# LangChain Agent with Qiskit IBM Runtime MCP Server

This notebook demonstrates how to create an AI agent using LangChain that connects to the **qiskit-ibm-runtime-mcp-server** via the Model Context Protocol (MCP).

The agent can interact with IBM Quantum services to:
- List available quantum backends
- Find the least busy backend
- Get detailed backend properties and calibration data
- Manage quantum jobs

## Architecture

```
┌─────────────┐     MCP Protocol     ┌──────────────────────────────────┐
│  LangChain  │ ◄──────────────────► │ qiskit-ibm-runtime-mcp-server    │
│    Agent    │                      │                                  │
└─────────────┘                      │  ┌────────────────────────────┐  │
                                     │  │   qiskit-ibm-runtime       │  │
                                     │  └────────────────────────────┘  │
                                     │               │                  │
                                     └───────────────│──────────────────┘
                                                     ▼
                                            ┌─────────────────┐
                                            │  IBM Quantum    │
                                            │    Cloud        │
                                            └─────────────────┘
```

## Setup

### 1. Install Dependencies

Run these commands in your terminal:

```bash
# Install the MCP server with examples dependencies
pip install qiskit-ibm-runtime-mcp-server[examples]

# Install your preferred LLM provider (choose one):
pip install langchain-openai       # For OpenAI
pip install langchain-anthropic    # For Anthropic Claude
pip install langchain-google-genai # For Google Gemini
pip install langchain-ollama       # For local Ollama
pip install langchain-ibm          # For IBM Watsonx
```

### 2. Configure Environment Variables

Set your IBM Quantum token and (optionally) instance for faster startup.

You can either:
- Set them in a `.env` file in this directory
- Set them as environment variables
- Enter them in the cell below

In [1]:
import os

from dotenv import load_dotenv

# LangChain imports
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools


# Load from .env file if it exists
load_dotenv()

# Or set directly (uncomment and fill in):
# os.environ["QISKIT_IBM_TOKEN"] = "your-ibm-quantum-token"
# os.environ["QISKIT_IBM_RUNTIME_MCP_INSTANCE"] = "your-instance-name"  # Optional but recommended for faster startup

# Set your LLM provider API key (uncomment the one you're using):
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key"
# os.environ["GOOGLE_API_KEY"] = "your-google-api-key"

# Verify configuration
print("Configuration status:")
print(f"  QISKIT_IBM_TOKEN: {'✓ Set' if os.getenv('QISKIT_IBM_TOKEN') else '✗ Not set'}")
print(
    f"  QISKIT_IBM_RUNTIME_MCP_INSTANCE: {'✓ Set' if os.getenv('QISKIT_IBM_RUNTIME_MCP_INSTANCE') else '✗ Not set (slower startup)'}"
)

Configuration status:
  QISKIT_IBM_TOKEN: ✓ Set
  QISKIT_IBM_RUNTIME_MCP_INSTANCE: ✓ Set


### 3. Find Your IBM Quantum Instance (Optional but Recommended)

Setting `QISKIT_IBM_RUNTIME_MCP_INSTANCE` significantly speeds up startup (from ~30 seconds to ~2 seconds).

Run this cell to find your available instances:

In [ ]:
# Uncomment to find your available instances
# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService()
# print("Available instances:")
# for instance in service.instances():
#     print(f"  - {instance['name']} ({instance['plan']})")

## Choose Your LLM Provider

Run **one** of the following cells based on your preferred LLM provider:

In [ ]:
# Option 1: OpenAI
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("Using OpenAI GPT-4o")

In [2]:
# Option 2: Anthropic Claude
from langchain_anthropic import ChatAnthropic


llm = ChatAnthropic(model="aws/claude-sonnet-4-5", temperature=0)
print("Using Anthropic Claude Sonnet")

Using Anthropic Claude Sonnet


In [ ]:
# Option 3: Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)
print("Using Google Gemini Pro")

In [ ]:
# Option 4: Local Ollama (no API key needed)
from langchain_ollama import ChatOllama


llm = ChatOllama(model="llama3.2", temperature=0)
print("Using local Ollama with Llama 3.2")

In [ ]:
# Option 5: IBM Watsonx
from langchain_ibm import ChatWatsonx


llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url=os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    params={"temperature": 0, "max_tokens": 4096},
)
print("Using IBM Watsonx Granite")

## Define the System Prompt

This prompt tells the agent what it can do and how to behave:

In [3]:
SYSTEM_PROMPT = """You are a helpful quantum computing assistant with access to IBM Quantum services
through the Qiskit IBM Runtime MCP server.

You can help users:
- Set up their IBM Quantum account (setup_ibm_quantum_account_tool)
- List available quantum backends (list_backends_tool)
- Find the least busy backend for running jobs (least_busy_backend_tool)
- Get detailed backend properties (get_backend_properties_tool)
- Get backend calibration data including T1, T2, error rates, and faulty qubits (get_backend_calibration_tool)
- List recent jobs (list_my_jobs_tool)
- Check job status (get_job_status_tool)
- Cancel jobs (cancel_job_tool)

Always provide clear explanations about quantum computing concepts when relevant.
When listing backends, highlight key properties like qubit count and operational status.
When showing calibration data, highlight faulty qubits/gates that users should avoid.
If an operation fails, explain the error and suggest possible solutions."""

## Create the MCP Client

This configures the connection to the qiskit-ibm-runtime-mcp-server:

In [4]:
def get_mcp_client():
    """Create and return an MCP client configured for the Qiskit IBM Runtime server."""
    return MultiServerMCPClient(
        {
            "qiskit-ibm-runtime": {
                "transport": "stdio",
                "command": "qiskit-ibm-runtime-mcp-server",
                "args": [],
                "env": {
                    "QISKIT_IBM_TOKEN": os.getenv("QISKIT_IBM_TOKEN", ""),
                    "QISKIT_IBM_RUNTIME_MCP_INSTANCE": os.getenv(
                        "QISKIT_IBM_RUNTIME_MCP_INSTANCE", ""
                    ),
                },
            }
        }
    )

## Create the Agent

Now we'll create a function that sets up the agent with a persistent MCP session.

Using a persistent session is important because it:
- Keeps a single MCP server process running
- Reuses the IBM Quantum service connection
- Makes tool calls much faster

In [5]:
async def create_agent_with_session(session):
    """Create a LangChain agent using an existing MCP session."""
    # Load tools from the existing session
    tools = await load_mcp_tools(session)
    print(f"Loaded {len(tools)} tools from MCP server:")
    for tool in tools:
        print(f"  - {tool.name}")

    # Create the agent using LangChain's create_agent
    agent = create_agent(llm, tools, system_prompt=SYSTEM_PROMPT)
    return agent

## Helper Function to Run Queries

This function sends a query to the agent and returns the response:

In [6]:
async def ask_agent(agent, query: str) -> str:
    """Send a query to the agent and return the response."""
    result = await agent.ainvoke({"messages": [HumanMessage(content=query)]})
    messages = result.get("messages", [])
    if messages:
        return messages[-1].content
    return "No response generated."

## Run the Agent

Now let's create the agent and ask it some questions!

The following cell starts the MCP server, creates the agent, and keeps the session open for multiple queries:

In [7]:
# Create MCP client and start a persistent session
mcp_client = get_mcp_client()

print("Starting MCP server and creating agent...")
print("(This may take a few seconds on first run)\n")

Starting MCP server and creating agent...
(This may take a few seconds on first run)



### Example 1: List Available Backends

Let's ask the agent to show us what quantum backends are available:

In [8]:
async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "What quantum backends are available?")
    print(response)

Loaded 8 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - list_backends_tool
  - least_busy_backend_tool
  - get_backend_properties_tool
  - get_backend_calibration_tool
  - list_my_jobs_tool
  - get_job_status_tool
  - cancel_job_tool
Great! Here are the **5 quantum backends** currently available:

## Available IBM Quantum Backends

| Backend | Qubits | Status | Pending Jobs | Type |
|---------|--------|--------|--------------|------|
| **ibm_fez** | 156 | ✅ Operational | 95 | Real quantum hardware |
| **ibm_pittsburgh** | 156 | ✅ Operational | 1,967 | Real quantum hardware |
| **ibm_torino** | 133 | ✅ Operational | 0 | Real quantum hardware |
| **ibm_kingston** | 156 | ✅ Operational | 89 | Real quantum hardware |
| **ibm_marrakesh** | 156 | ✅ Operational | 9,600 | Real quantum hardware |

### Key Observations:

- **All backends are operational** and ready to accept jobs
- **ibm_torino** has **no pending jobs** - this would be the fastest to run on right now!
- **ibm_fez*

### Example 2: Find the Least Busy Backend

When you want to run a job quickly, you want the backend with the shortest queue:

In [9]:
async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "Which backend has the shortest queue right now?")
    print(response)

Loaded 8 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - list_backends_tool
  - least_busy_backend_tool
  - get_backend_properties_tool
  - get_backend_calibration_tool
  - list_my_jobs_tool
  - get_job_status_tool
  - cancel_job_tool
Great news! The **ibm_torino** backend currently has the shortest queue with:

- **0 pending jobs** - no wait time!
- **133 qubits** available
- **Status**: Operational and active

This is an excellent time to run quantum circuits on ibm_torino if you have any jobs ready to submit. The lack of pending jobs means your circuits would start executing almost immediately.

Would you like to know more details about this backend, such as its calibration data, gate fidelities, or other properties?


### Example 3: Get Backend Details

Let's get detailed information about a specific backend:

In [10]:
async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(
        agent, "Tell me about the ibm_fez backend, including its calibration data"
    )
    print(response)

Loaded 8 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - list_backends_tool
  - least_busy_backend_tool
  - get_backend_properties_tool
  - get_backend_calibration_tool
  - list_my_jobs_tool
  - get_job_status_tool
  - cancel_job_tool
## IBM Fez Backend Overview

**ibm_fez** is a large-scale quantum processor with impressive specifications:

### General Properties
- **Processor Type**: Heron r2 (IBM's latest generation)
- **Number of Qubits**: 156 qubits
- **Status**: ✅ Operational (active)
- **Pending Jobs**: 95 jobs in queue
- **Backend Version**: 1.3.29

### Gate Set & Operations
- **Basis Gates**: `cz`, `id`, `rz`, `sx`, `x`
  - Uses CZ (controlled-Z) gates instead of CNOT for two-qubit operations
  - Single-qubit gates: X (bit flip), SX (√X), RZ (Z-rotation), ID (identity)
- **Max Shots**: 100,000 per circuit
- **Max Experiments**: 300 circuits per job

### Connectivity
The backend has a heavy-hex lattice topology with 156 qubits arranged in a grid pattern. Each qubi

### Example 4: Check Recent Jobs

See what quantum jobs you've submitted recently:

In [11]:
async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "Show me my recent quantum jobs")
    print(response)

Loaded 8 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - list_backends_tool
  - least_busy_backend_tool
  - get_backend_properties_tool
  - get_backend_calibration_tool
  - list_my_jobs_tool
  - get_job_status_tool
  - cancel_job_tool
Great! Here are your 10 most recent quantum jobs:

| Job ID | Status | Backend | Creation Date |
|--------|--------|---------|---------------|
| d4uigskgk3fc73au0sig | ✅ DONE | ibm_fez | Dec 13, 2025 09:48:50 |
| d4uign4gk3fc73au0scg | ✅ DONE | ibm_fez | Dec 13, 2025 09:48:28 |
| d4uig0sgk3fc73au0rgg | ✅ DONE | ibm_fez | Dec 13, 2025 09:46:59 |
| d4uiflkgk3fc73au0r0g | ✅ DONE | ibm_fez | Dec 13, 2025 09:46:14 |
| d4uifj7g0u6s73d9qjcg | ✅ DONE | ibm_fez | Dec 13, 2025 09:46:04 |
| d4uifgeaec6c738rdifg | ✅ DONE | ibm_fez | Dec 13, 2025 09:45:53 |
| d4ui9gleastc73ch8f1g | ✅ DONE | ibm_fez | Dec 13, 2025 09:33:06 |
| d4ui3muaec6c738rd62g | ✅ DONE | ibm_fez | Dec 13, 2025 09:20:43 |
| d4ui3h7g0u6s73d9q6ig | ✅ DONE | ibm_fez | Dec 13, 2025 09:20:2

### Example 5: Interactive Chat

Run this cell to have an interactive conversation with the agent:

In [12]:
async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)
    print("Agent ready! Type your questions below.")
    print("Enter 'quit' to stop.\n")

    while True:
        try:
            query = input("You: ").strip()
            if not query:
                continue
            if query.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            response = await ask_agent(agent, query)
            print(f"\nAssistant: {response}\n")
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break

Loaded 8 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - list_backends_tool
  - least_busy_backend_tool
  - get_backend_properties_tool
  - get_backend_calibration_tool
  - list_my_jobs_tool
  - get_job_status_tool
  - cancel_job_tool
Agent ready! Type your questions below.
Enter 'quit' to stop.


Assistant: Hello! 👋 I'm your quantum computing assistant with access to IBM Quantum services through Qiskit.

I can help you with various tasks related to IBM Quantum computers, including:

- **Account Setup**: Connect to your IBM Quantum account
- **Backend Information**: Browse available quantum computers and simulators
- **System Status**: Check which systems are operational and least busy
- **Hardware Details**: Get detailed properties like qubit count, connectivity, gate sets, and calibration data (T1/T2 times, error rates, faulty qubits)
- **Job Management**: View your recent jobs, check their status, or cancel them

What would you like to do today? For example, I can:
- S

## Custom Queries

Use this cell to ask the agent any question:

In [13]:
# Enter your question here:
MY_QUESTION = "Compare the ibm_fez and ibm_marrakesh backends"

async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)
    response = await ask_agent(agent, MY_QUESTION)
    print(response)

Loaded 8 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - list_backends_tool
  - least_busy_backend_tool
  - get_backend_properties_tool
  - get_backend_calibration_tool
  - list_my_jobs_tool
  - get_job_status_tool
  - cancel_job_tool
## Comparison: IBM Fez vs IBM Marrakesh

Both backends are **156-qubit Heron r2 processors** with identical architecture, but they differ in availability and calibration quality. Here's a detailed comparison:

### **Basic Properties**
| Property | IBM Fez | IBM Marrakesh |
|----------|---------|---------------|
| **Qubits** | 156 | 156 |
| **Processor Type** | Heron r2 | Heron r2 |
| **Status** | ✅ Operational | ✅ Operational |
| **Pending Jobs** | 95 | 9,600 |
| **Backend Version** | 1.3.29 | 1.0.18 |
| **Basis Gates** | cz, id, rz, sx, x | cz, id, rz, sx, x |
| **Max Shots** | 100,000 | 100,000 |
| **Max Experiments** | 300 | 300 |

### **🎯 Key Difference: Availability**
- **IBM Fez**: Only **95 pending jobs** - **Much less busy!** ✨
- **I

## Available Tools

The agent has access to these tools provided by the MCP server:

| Tool | Description |
|------|-------------|
| `setup_ibm_quantum_account_tool` | Set up IBM Quantum account with credentials |
| `list_backends_tool` | List all available quantum backends |
| `least_busy_backend_tool` | Find the least busy operational backend |
| `get_backend_properties_tool` | Get detailed properties of a specific backend |
| `get_backend_calibration_tool` | Get calibration data (T1, T2, error rates, faulty qubits) |
| `list_my_jobs_tool` | List user's recent quantum jobs |
| `get_job_status_tool` | Get status of a specific job |
| `cancel_job_tool` | Cancel a running or queued job |

## Troubleshooting

### Slow startup?
Set `QISKIT_IBM_RUNTIME_MCP_INSTANCE` environment variable to skip instance lookup.

### Authentication errors?
Verify your `QISKIT_IBM_TOKEN` is correct and your IBM Quantum account is active.

### MCP server not found?
Make sure `qiskit-ibm-runtime-mcp-server` is installed: `pip install qiskit-ibm-runtime-mcp-server[examples]`